# Usar um pipeline para ler e processar um conjunto de ficheiros e usar esse mesmo pipeline como 'conjunto de treino' #

In [117]:
import glob
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers


plt.style.use('seaborn-poster')
%matplotlib inline

A api, tf.data.Dataset , permite criar um pipeline de leitura de ficheiros e seu eventual processamento, usando a função 'map':
https://www.tensorflow.org/guide/data
,
https://www.tensorflow.org/guide/data_performance
.

Podemos fornecer diretamente um tfdata.Dataset ao método fit é uma forma muito eficiente de efetuar o treino de um modelo keras: https://www.tensorflow.org/guide/keras/train_and_evaluate#training_evaluation_from_tfdata_datasets


No que se segue exemplificaremos a utilização desta api num conjunto de ficheiros do 2ª trabalho. O tf.data.Dataset permite criar 'batches' que podem sevir de inputs e labels a um modelo keras que faça a segmentação dos sons cardíacos.

In [74]:
dir = './data/validacaoAlunos/'
list_npy_files = glob.glob(dir + '*.npy')
fnames_dataset = tf.data.Dataset.from_tensor_slices(list_npy_files)

In [75]:
print(fnames_dataset)

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>


Podemos aceder um a um aos elementos (ou batches de elementos) de um dataset

In [76]:
it = iter(fnames_dataset)
ex = next(it)
ex

<tf.Tensor: shape=(), dtype=string, numpy=b'./data/validacaoAlunos\\14998_PV_sigTarg.npy'>

A partir deste dataset inicial, fnames_dataset, vamos criar um cujo conteudo é o resultado de ler cada um dos ficheiros npy.

Começamos por uma função auxiliar que vai ser usada pela função 'map' para ler o conteúdo de cada um dos ficheiros do fnames_dataset e criar um novo dataset com estes conteúdos.

In [77]:
def read_npy(fname):
    """fname should be a npy file"""
    fname = fname.decode()
    recData = np.load(fname)
    return recData.astype(np.float32)

In [78]:
dataset_raw = fnames_dataset.map(lambda x: tf.numpy_function(read_npy, [x], [tf.float32]))

Poderiamos também usar a função 'map' para separar inputs e targets e obter um dataset que produz um 'tuple', (input, target) 

In [79]:
dataset2x = dataset_raw.map(lambda x: (x[0],x[1]))

((<tf.Tensor: shape=(), dtype=int32, numpy=25435>,
  <tf.Tensor: shape=(197, 129), dtype=complex64, numpy=
  array([[-9.2504510e+02+0.00000000e+00j, -2.5206216e+03+9.34842773e+03j,
           7.7033047e+03-2.93779541e+03j, ...,
          -1.2470239e+02-3.29393311e+01j, -1.8242554e+01-1.16825195e+02j,
           2.4242975e+02+0.00000000e+00j],
         [ 8.4404658e+03+0.00000000e+00j,  3.1762125e+04+1.85271816e+04j,
          -7.7123828e+04+4.90774531e+04j, ...,
          -2.7286328e+02-1.40541016e+02j, -4.6508691e+02-9.62832031e+01j,
           1.0056179e+03+0.00000000e+00j],
         [ 1.3549773e+04+0.00000000e+00j,  1.6253398e+04-1.47566152e+04j,
          -5.3920125e+04-7.76776250e+04j, ...,
          -4.6308789e+02+2.09753906e+02j, -9.8410645e+01+8.70815430e+01j,
           6.1733789e+02+0.00000000e+00j],
         ...,
         [-1.0518837e+04+0.00000000e+00j,  8.6715977e+03-1.49277891e+04j,
           2.9549316e+03+1.47093398e+04j, ...,
          -2.1637695e+01-4.82548828e+01j,  2

O nosso dataset pode também produzir 'batches' em vez de exemplos individuais.  No nosso caso os exempos não tẽm todos o mesmo comprimento, assim, teremos de extender(por zeros??) os exemplos mais curtos de cada batch de modo a todos os exemplos do mesmo batch terem o mesmo comprimento.

    Poderiamos também usar a função map para aplicar a 'stft' aos inputs 

In [80]:
def randomExtract(x):
    length = tf.shape(x)[1]
    init = tf.random.uniform([], minval=0,maxval=length-L,dtype=tf.dtypes.int32)
    return x[:, init:init+L]

In [81]:
frame_length= 1000
frame_step = 10

def stft(x):
    return tf.signal.stft(
    x,
    frame_length,
    frame_step,
    fft_length=None,
    window_fn=tf.signal.hann_window,
    pad_end=False,
    name=None
)

In [82]:
dataset_stft = dataset_raw.map(lambda x: ((tf.shape(x)[1],tf.signal.stft(x[0], 256,128)),x[1]))

In [92]:
print(list(dataset_stft)[0][0][0])
print(list(dataset_stft)[0][0][1].shape)
print(list(dataset_stft)[1][0][0])
print(list(dataset_stft)[1][0][1].shape)

tf.Tensor(25435, shape=(), dtype=int32)
(197, 129)
tf.Tensor(26714, shape=(), dtype=int32)
(207, 129)


In [94]:
batch_size = 2
k=129
dataset_pb = dataset_stft.padded_batch(batch_size, padded_shapes=(([], tf.TensorShape([None, k])),[None,]))

it3 = iter(dataset_pb)
ex3 = next(it3)
ex3

((<tf.Tensor: shape=(2,), dtype=int32, numpy=array([25435, 26714])>,
  <tf.Tensor: shape=(2, 207, 129), dtype=complex64, numpy=
  array([[[-9.2504510e+02+0.00000000e+00j, -2.5206216e+03+9.34842773e+03j,
            7.7033047e+03-2.93779541e+03j, ...,
           -1.2470239e+02-3.29393311e+01j, -1.8242554e+01-1.16825195e+02j,
            2.4242975e+02+0.00000000e+00j],
          [ 8.4404658e+03+0.00000000e+00j,  3.1762125e+04+1.85271816e+04j,
           -7.7123828e+04+4.90774531e+04j, ...,
           -2.7286328e+02-1.40541016e+02j, -4.6508691e+02-9.62832031e+01j,
            1.0056179e+03+0.00000000e+00j],
          [ 1.3549773e+04+0.00000000e+00j,  1.6253398e+04-1.47566152e+04j,
           -5.3920125e+04-7.76776250e+04j, ...,
           -4.6308789e+02+2.09753906e+02j, -9.8410645e+01+8.70815430e+01j,
            6.1733789e+02+0.00000000e+00j],
          ...,
          [ 0.0000000e+00+0.00000000e+00j,  0.0000000e+00+0.00000000e+00j,
            0.0000000e+00+0.00000000e+00j, ...,
        

In [115]:
print(ex3[1].shape)

(2, 26714)


In [85]:
print(list(dataset3)[0][0][1].shape)
print(list(dataset3)[1][0][1].shape)


(2, 207, 129)
(2, 240, 129)


In [86]:
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])

In [120]:
class PositionEmbedding(layers.Layer):
    def __init__(self, maxlen, embed_dim):
        super(PositionEmbedding, self).__init__()
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[1]
        batchS = tf.shape(x)[0]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        positions = tf.expand_dims(positions, axis = 0)
        positions = tf.repeat(positions, batchS, axis=0)
        return x + positions

In [121]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [126]:
inputA = keras.Input(shape=([]))
inputB = keras.Input(shape=([None, 129]))
embDim = 10

#processa o input b e faz o upsampling e corta ao tamanho do target

newEmb = layers.TimeDistributed(layers.Dense(embDim))(inputB)
inputsPosEmb = PositionEmbedding(maxlen=20, embed_dim=embDim)(newEmb)
transf1 = TransformerBlock(embed_dim=embDim,num_heads=3, ff_dim=8)(inputsPosEmb)
transf2 =TransformerBlock(embed_dim=embDim,num_heads=3, ff_dim=8)(transf1)
out = layers.TimeDistributed(layers.Dense(3))(transf2)
#TODO insert LSTM here?

transf_model = keras.Model(inputs=inputB, outputs=out)

In [108]:
print(list(dataset3)[0][0][1].shape)

(2, 207, 129)


In [129]:
o= transf_model(ex3[0][1])

InvalidArgumentError: Exception encountered when calling layer "embedding_4" (type Embedding).

indices[20] = 20 is not in [0, 20) [Op:ResourceGather]

Call arguments received by layer "embedding_4" (type Embedding):
  • inputs=tf.Tensor(shape=(207,), dtype=int32)

In [ ]:
o= model)

In [ ]:
lr = 1.0e-4
opt = tf.keras.optimizers.Adam(learning_rate=lr)
transf_model.compile(optimizer=opt, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    metrics = 'accuracy')
transf_model.fit(inputsData, targets, batch_size=4, epochs = 50)

sinal com tamanho inicial, 